In [ ]:
!pip install fosforml
!pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [ ]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

In [ ]:
df = 'ATTRITION_PREDICTIVE_DATA'

In [ ]:
sf_df = my_session.sql("select * from {}".format(df))

In [ ]:
import pandas as pd
pandas_df = sf_df.to_pandas()

In [ ]:
print(pandas_df.isnull().sum())

In [ ]:
Original_df = pandas_df.dropna()

In [ ]:
print(Original_df.isnull().sum())

In [ ]:
# Ensure date columns are parsed correctly
date_columns = ['SCHOOL_ENDDATE', 'JOB_STARTDATE', 'JOB_ENDDATE']
for col in date_columns:
    Original_df[col] = pd.to_datetime(Original_df[col], errors='coerce')

In [ ]:
# Display the cleaned DataFrame
print(Original_df.head())

In [ ]:
# Assuming 'CHURN' is your target variable and 'TENURE_MONTHS' is a feature
y = Original_df['CHURN']
X = Original_df[['DEGREE_CLEAN', 'ORGANIZATION_TYPE']]  # Add more features as needed

In [ ]:
# Add a constant to the features
import statsmodels.api as sm  # Import statsmodels
X = sm.add_constant(X)

In [ ]:
# Split data into training and testing sets
train_size = int(len(y) * 0.8)
y_train, y_test = y[:train_size], y[train_size:]
X_train, X_test = X[:train_size], X[train_size:]

In [ ]:
print(Original_df.dtypes)

In [ ]:
# Fit ARIMAX model
model = sm.tsa.ARIMA(y_train, exog=X_train, order=(1, 1, 1))
model_fit = model.fit()

In [ ]:
# Make predictions
predictions = model_fit.forecast(steps=len(y_test), exog=X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

In [ ]:
# Make predictions for the next 12 months
future_steps = 12
future_exog = X_test.iloc[-1:].append([X_test.iloc[-1:]] * (future_steps - 1), ignore_index=True)  # Adjust exogenous variables as needed
predictions = model_fit.forecast(steps=future_steps, exog=future_exog)

print(predictions)

In [ ]:
df = Original_df[["DISTANCE","DEGREE_CLEAN","ROLE","COMPANY_NAME","ORGANIZATION_TYPE","ORGANIZATION_OWNERSHIP","JOB_ENDDATE","CHURN"]]

In [ ]:
df

In [ ]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])

In [ ]:
df.set_index('JOB_ENDDATE', inplace = True)

In [ ]:
# Ensure the index is unique
df = df[~df.index.duplicated(keep='first')]

In [ ]:
# Group by categorical columns and resample monthly, then sum CHURN
New_df = df.groupby(["DISTANCE", "DEGREE_CLEAN", "ROLE", "COMPANY_NAME", "ORGANIZATION_TYPE", "ORGANIZATION_OWNERSHIP"]).resample('M').agg({'CHURN': 'sum'}).reset_index()

In [ ]:
# Filter rows where CHURN is greater than 0
df1 = New_df[New_df['CHURN'] > 0]

In [ ]:
df1

In [ ]:
# Set date column as index
df1.set_index('JOB_ENDDATE', inplace=True)

In [ ]:
# Ensure the index is unique
df1 = df1[~df1.index.duplicated(keep='first')]

In [ ]:
# Split data into training and testing sets
train = df1[:pd.Timestamp('2024-08-31')]
test = df1[pd.Timestamp('2024-09-01'):]

In [ ]:
# Ensure date column is in datetime format
df1['JOB_ENDDATE'] = pd.to_datetime(df1['JOB_ENDDATE'])

In [ ]:
# Ensure the index is unique
df = df[~df.index.duplicated(keep='first')]


In [ ]:
# Set date column as index
df1.set_index('JOB_ENDDATE', inplace=True)

In [ ]:
# Split data into training and testing sets
train = df1[:pd.Timestamp('2024-08-31')]
test = df1[pd.Timestamp('2024-09-01'):]

In [ ]:
# Split data into training and testing sets
train = df1[:'2024-08']
test = df1['2024-09':]

In [ ]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])
monthly_data = df.resample('M', on='JOB_ENDDATE').sum()

In [ ]:
monthly_data